<a href="https://colab.research.google.com/github/NikhilD2003/Medical-Report-Chatbot/blob/main/MedicalChatbotBufferMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain langchain-community faiss-cpu sentence-transformers docx2txt requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
import os
import requests
import docx2txt
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_core.language_models.chat_models import SimpleChatModel
from langchain_core.messages import HumanMessage
from typing import Optional
from pydantic import Field

class MistralOpenRouter(SimpleChatModel):
    api_key: str = Field(...)
    model: str = "mistralai/mistral-small-3.2-24b-instruct:free"
    base_url: str = "https://openrouter.ai/api/v1/chat/completions"

    def _call(self, messages, **kwargs) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://example.com",
            "X-Title": "Conversational RAG"
        }

        body = {
            "model": self.model,
            "messages": [
    {
        "role": "user" if m.__class__.__name__ == "HumanMessage"
        else "assistant" if m.__class__.__name__ == "AIMessage"
        else "system" if m.__class__.__name__ == "SystemMessage"
        else "user",
        "content": m.content
    } for m in messages
]
        }

        response = requests.post(self.base_url, headers=headers, json=body)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    @property
    def _llm_type(self) -> str:
        return "mistral-openrouter"


In [ ]:
def load_medical_report(file_path):
    if file_path.endswith(".docx"):
        text = docx2txt.process(file_path)
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
    elif file_path.endswith(".pdf"):
        import fitz
        text = ""
        with fitz.open(file_path) as doc:
            for page in doc:
                text += page.get_text()
    else:
        raise ValueError("Only .txt, .docx, or .pdf supported.")
    return [Document(page_content=text)]

In [ ]:
def create_vectorstore(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=60, chunk_overlap=30)
    chunks = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(chunks, embedding=embeddings)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def start_conversational_rag(vectorstore):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    llm = MistralOpenRouter(api_key=OPENROUTER_API_KEY)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        verbose=True
    )
    return chain

In [ ]:
if __name__ == "__main__":
    OPENROUTER_API_KEY = "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"
    file_path = "/content/Med.docx"  # your file here

    docs = load_medical_report(file_path)
    vectorstore = create_vectorstore(docs)
    chat_chain = start_conversational_rag(vectorstore)

    print("🤖 Chatbot ready. Ask medical questions.")

    while True:
        query = input("\nYou: ")
        if query.lower() in ["exit", "quit"]:
            break
        response = chat_chain.run(query)
        print("\nBot:", response)

🤖 Chatbot ready. Ask medical questions.

You: What is the primary diagnosis?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Diagnosis

Primary Diagnosis: Asthma

Secondary Diagnosis: Seasonal Allergies

Medical Report for School

Student Information

Recent Symptoms: Occasional shortness of breath during
Human: What is the primary diagnosis?

> Finished chain.

> Finished chain.

Bot: The primary diagnosis is Asthma.

You: how can i treat it?


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the primary diagnosis?
Assistant: The primary diagnosis is Asthma.
Follo